In [187]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

In [148]:
train= pd.read_csv("train (1).csv")
test = pd.read_csv("test (1).csv",names = ["Tweets","label"])

In [149]:
train

,Tweets,label
0,Asked #ChatGPT about what it thinks are the pr...,Positive
1,#ChatGPT tornado has already traveled around t...,Neutral
2,This is a great explanation of why #EVs are mo...,Positive
3,‘if you need to write a box-ticking social med...,Positive
4,Just saw an AI tool making my coffee for me. \...,Positive
...,...,...
765,NaN,NaN
766,NaN,NaN
767,NaN,NaN
768,NaN,NaN


In [150]:
test


,Tweets,label
0,@CoffeeVectors @jenny____r confirmed #ChatGPT ...,Positive
1,Sooooo depressing (our competition as journos ...,Negative
2,I just had a 4 hour deep conversation with #Ch...,Positive
3,I began to explore #chatgpt today.\n\nI asked ...,Negative
4,/2023\n1. Ask #ChatGPT\n2. Add some content\n3...,Positive
...,...,...
166,Earth's #OpenAIChatGPT has become sentient an...,Positive
167,The first response was elaborate with some obv...,Positive
168,The absolutely terrifying inevitability of #Ch...,Negative
169,I asked ChatGPT what video would give the most...,Neutral


In [151]:
train.head()

,Tweets,label
0,Asked #ChatGPT about what it thinks are the pr...,Positive
1,#ChatGPT tornado has already traveled around t...,Neutral
2,This is a great explanation of why #EVs are mo...,Positive
3,‘if you need to write a box-ticking social med...,Positive
4,Just saw an AI tool making my coffee for me. \...,Positive


In [152]:
train.isnull().sum()


Tweets    171
label     171
dtype: int64

In [73]:
test.isnull().sum()

Tweets    0
label     0
dtype: int64

In [153]:
train=train.dropna()

In [154]:
train["label"].unique()

array(['Positive', 'Neutral', 'Negative'], dtype=object)

In [155]:
train["label"]=train["label"].map({"Positive":0,"Neutral":1,"Negative":2})

<ipython-input-155-3db99b808878>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["label"]=train["label"].map({"Positive":0,"Neutral":1,"Negative":2})


In [156]:
test["label"]=test["label"].map({"Positive":0,"Neutral":1,"Negative":2})

In [157]:
train["label"].unique()

array([0, 1, 2])

In [158]:
import pandas as pd
import re

def remove(text):
    if isinstance(text, str):
        return re.sub(r'[^a-zA-Z0-9\s]', '', text)
    else:
        return str(text)
train['Tweets'] = train['Tweets'].apply(remove)
test["Tweets"] = test["Tweets"].apply(remove)



<ipython-input-158-8ad02234800b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Tweets'] = train['Tweets'].apply(remove)


In [159]:
x_train=train["Tweets"]
x_test=test["Tweets"]

In [160]:
y_train=train["label"]
y_test = test["label"]

In [87]:
x_train


0      Asked ChatGPT about what it thinks are the pro...
1      ChatGPT tornado has already traveled around th...
2      This is a great explanation of why EVs are mor...
3      if you need to write a boxticking social media...
4      Just saw an AI tool making my coffee for me \n...
                             ...                        
594    4 GPTs AI capabilities are also more accessibl...
595    3 KeyboardX also provides a more secure enviro...
596    2 KeyboardX allows users to quickly and easily...
597    I asked ChatGPT why the app Im working on Keyb...
598    Theres been a lot of talk about AI engines and...
Name: Tweets, Length: 599, dtype: object

In [88]:
y_train

0      0
1      1
2      0
3      0
4      0
      ..
594    0
595    0
596    0
597    0
598    0
Name: label, Length: 599, dtype: int64

In [89]:
tokeniser = tf.keras.preprocessing.text.Tokenizer()
tokensizer = tokeniser.fit_on_texts(x_train)
encode_train = tokeniser.texts_to_sequences(x_train)
encode_test = tokeniser.texts_to_sequences(x_test)


In [174]:
max_length = 10
padded_train = tf.keras.preprocessing.sequence.pad_sequences(encode_train,maxlen=max_length,padding="post")
padded_test = tf.keras.preprocessing.sequence.pad_sequences(encode_test,maxlen=max_length,padding="post")

In [212]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))

In [200]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 20, 100)           1000000   
                                                                 
 lstm_8 (LSTM)               (None, 128)               117248    
                                                                 
 dense_26 (Dense)            (None, 3)                 387       
                                                                 
Total params: 1117635 (4.26 MB)
Trainable params: 1117635 (4.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [215]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', mode='min', patience=10)

In [216]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [166]:
x_train_pad = pad_sequences(encode_train, maxlen=max_sequence_length)
x_test_pad = pad_sequences(encode_test, maxlen=max_sequence_length)

In [217]:
model.fit(padded_train, y_train, validation_data=(padded_test, y_test), epochs=10,callbacks=[early_stop])


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


19/19 [==============================] - 2s 27ms/step - loss: 0.0000e+00 - accuracy: 0.5960 - val_loss: 0.0000e+00 - val_accuracy: 0.6725
Epoch 2/10
19/19 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 0.6311 - val_loss: 0.0000e+00 - val_accuracy: 0.6725
Epoch 3/10
19/19 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 0.6311 - val_loss: 0.0000e+00 - val_accuracy: 0.6725
Epoch 4/10
19/19 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 0.6311 - val_loss: 0.0000e+00 - val_accuracy: 0.6725
Epoch 5/10
19/19 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 0.6311 - val_loss: 0.0000e+00 - val_accuracy: 0.6725
Epoch 6/10
19/19 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 0.6311 - val_loss: 0.0000e+00 - val_accuracy: 0.6725
Epoch 7/10
19/19 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 0.6311 - val_loss: 0.

In [209]:
def c_report(y_true, y_pred):
   print("Classification Report")
   print(classification_report(y_true, y_pred))
   acc_sc = accuracy_score(y_true, y_pred)
   print(f"Accuracy : {str(round(acc_sc,2)*100)}")
   return acc_sc




In [219]:

preds = (model.predict(padded_test) > 0.5).astype("int32")
c_report(y_test, preds)



6/6 [==============================] - 0s 3ms/step
Classification Report
              precision    recall  f1-score   support

           0       0.67      1.00      0.80       115
           1       0.00      0.00      0.00        19
           2       0.00      0.00      0.00        37

    accuracy                           0.67       171
   macro avg       0.22      0.33      0.27       171
weighted avg       0.45      0.67      0.54       171

Accuracy : 67.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.672514619883041